In [139]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import pandas as pd

vis_data = pd.read_csv("./train.csv", encoding = 'ISO-8859-1', low_memory = False)

In [140]:
data = vis_data[['fine_amount', 'state_fee', 'late_fee', 'discount_amount', 'balance_due', 'compliance']].dropna()

In [141]:
values = data['compliance'].value_counts().to_dict()
c_name, c_cnt = sorted(values.items(), key=lambda x: x[1])[0]
data_2 = pd.concat([data.query('compliance == @c_name'), data.query('compliance != @c_name').iloc[:c_cnt]],
                   ignore_index=True, sort=False)


data_2.head()

,fine_amount,state_fee,late_fee,discount_amount,balance_due,compliance
0,750.0,10.0,75.0,0.0,75.0,1.0
1,100.0,10.0,10.0,0.0,10.0,1.0
2,100.0,10.0,10.0,0.0,10.0,1.0
3,250.0,10.0,25.0,0.0,25.0,1.0
4,250.0,10.0,0.0,25.0,0.0,1.0


In [142]:
def run(data):
    X = data[list(set(data.columns) - set(['compliance']))]
    y = data['compliance']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True)
    model = DecisionTreeClassifier(random_state=23)
    model.fit(X_train, y_train)

    return f1_score(y_test, model.predict(X_test))

In [143]:
result = run(data_2) - run(data)

In [144]:
print(result)

0.29706745916374333


In [145]:
vis_data_small = vis_data[['fine_amount', 'state_fee', 'late_fee',
                           'discount_amount', 'balance_due', 'compliance']].dropna()
X = vis_data_small[['fine_amount', 'state_fee', 'late_fee', 'discount_amount', 'balance_due']]
y = vis_data_small['compliance']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)
clf = DecisionTreeClassifier(random_state=23)
clf.fit(X_train, y_train)
f1_imbalanced = f1_score(y_test, clf.predict(X_test))

zeros = y[y == 0.0]
ones = y[y == 1.0]
down_index = list(zeros.index[:len(ones)]) + list(ones.index)
X_down = X.iloc[down_index, :]
y_down = y[down_index]
X_train, X_test, y_train, y_test = train_test_split(X_down, y_down, test_size=0.3, shuffle=False)
clf = DecisionTreeClassifier(random_state=23)
clf.fit(X_train, y_train)
f1_balanced = f1_score(y_test, clf.predict(X_test))

result = f1_balanced - f1_imbalanced

print(result)


IndexError: positional indexers are out-of-bounds